In [3]:
import dotenv
import os

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("GPT_KEY")

In [2]:
# https://python.langchain.com/docs/expression_language/get_started/



from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = OpenAI(model="gpt-3.5-turbo-instruct")

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
prompt_value = prompt.invoke({"topic": "ice cream"})
output_parser = StrOutputParser()


llm.invoke(prompt_value)
# chain.invoke({"topic": "ice cream"})

'\n\nWhy did the ice cream go to therapy?\n\nBecause it had a rocky road.'

In [4]:
# https://python.langchain.com/docs/expression_language/get_started/


from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")


from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Escolha somente uma palavra que defina a emoção predominante de cada uma das frases a seguir: '{teste1}', '{teste2}', '{teste3}'")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

saida = chain.invoke({"teste1": "Minha casa pegou fogo semana passada, e perdi todo meu dinheiro... Não sei o que fazer", "teste2": "Estou com muita vontade de comer um peru assado de natal.", "teste3":"Minha namorada é muito bonita e inteligente! Sou muito sortudo de ter ela ao meu lado!"})

In [5]:
!pip install "langchain[docarray]"

  Using cached docarray-0.32.1-py3-none-any.whl (215 kB)
  Using cached types_requests-2.32.0.20240521-py3-none-any.whl (15 kB)
  Using cached protobuf-5.26.1-cp310-abi3-win_amd64.whl (420 kB)
  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build hnswlib


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\jalfr\Desktop\ai-startup\simulai\env\Scripts\python.exe' 'C:\Users\jalfr\Desktop\ai-startup\simulai\env\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\jalfr\AppData\Local\Temp\tmpkf9yy87n'
       cwd: C:\Users\jalfr\AppData\Local\Temp\pip-install-cibbb0v6\hnswlib_ca878e0c4fae482599c7e045553728c5
  Complete output (5 lines):
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  ----------------------------------------
  ERROR: Failed building wheel for hnswlib
ERROR: Could not build wheels for hnswlib which use PEP 517 and cannot be installed directly
You should consider upgrading via the 'C:\Users\jalfr\Desktop\ai-startup\simulai\env\Scripts\python.exe -m pip install --upgrade pip' com

In [4]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["Pedro Pertusi é amigo do Alfredo", "Todo amigo dos amigos do Pertusi é também amigo dele"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

# https://js.langchain.com/docs/integrations/vectorstores/hnswlib

chain.invoke("Se Alfredo é amigo de Eduardo, quem são os amigos de Pertusi?")

ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.

In [90]:
import dotenv
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from typing import List

import pandas as pd

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("GPT_KEY")

# https://python.langchain.com/docs/expression_language/get_started/


MODEL = ChatOpenAI(model="gpt-4")


def fetchData(message:str, id:int) -> str:

    df_messages = pd.read_csv("../lang-processing/data/transcripts.csv", sep=";", encoding="UTF-8")

    context = df_messages[df_messages['CONVERSATION_ID'] == id]

    id_script = context['ROTEIRO_ID'].iloc[0]

    df_script = pd.read_csv("../lang-processing/data/roteiros.csv", sep=";", encoding="UTF-8")

    df_script = df_script[df_script['ROTEIRO_ID'] == id_script]

    prompt = buildPrompt(context=context, df_scripts=df_script.iloc[0], message=message)

    stage = context['STAGE'].max()+1
    m_id = context['MENSAGEM_ID'].max()


    df_script_stages = pd.read_csv("../lang-processing/data/roteiro_stages.csv", sep=";", encoding="UTF-8")

    df_script_stages = df_script_stages[df_script_stages['ROTEIRO_ID'] == id_script]

    options = df_script_stages[df_script_stages['STAGE'] == stage]
    options = options['OPTION'].astype(str).to_list()

    response = getResponse(options=options, prpt=prompt)

    print(response)
    print(options[response-1])

    data = {
        "CONVERSATION_ID" : [id, id], 
        "ROTEIRO_ID" : [id_script, id_script], 
        "STAGE" : [stage, stage], 
        "MENSAGEM_ID" : [m_id+1, m_id+2], 
        "TRANSCRIPT" : [message, options[response-1]], 
        "SENDER": ["USER", "CHAT"]
    }


    df = pd.DataFrame(data)

    print(df.head())
    
    df.to_csv('../lang-processing/data/transcripts.csv', mode='a', index=False, header=False, sep=";")

    return response



def buildPrompt(context: pd.DataFrame, df_scripts: pd.Series, message:str):

    prompt = df_scripts["CONTEXT"] + "\n\n Essa é a conversa até agora: \n"

    messages = context['TRANSCRIPT'].to_list()
    senders = context['SENDER'].to_list()


    for i in range(len(context)):
        prompt += df_scripts[senders[i]] + " - " + messages[i] + "\n\n"

    prompt += "A última mensagem do/a " + df_scripts["USER"] + " foi:\n" + message

    prompt+= '''
Qual seria a próxima resposta mais apropriada, entre as seguintes:

    {Resposta1}

    {Resposta2}

    {Resposta3}

Responda somente com um número, correspondente a resposta escolhida
    '''

    return prompt


def getResponse(options: List[str], prpt: str) -> int:

    prompt = ChatPromptTemplate.from_template(prpt)
    output_parser = StrOutputParser()

    chain = prompt | MODEL | output_parser

    n = chain.invoke({f"Resposta{i+1}":options[i] for i in range(len(options))})

    return int(n)

In [91]:
fetchData("HIV pode ser transmitido de várias formas Marcia, apesar de sexualmente ser a principal forma. A senhora compartilha algum item dométisco com algum possível portador de HIV?", 1)

2
 "Eu não tenho nenhum tipo de contato com pessoas com HIV"
   CONVERSATION_ID  ROTEIRO_ID  STAGE  MENSAGEM_ID  \
0                1           1      2            3   
1                1           1      2            4   

                                          TRANSCRIPT SENDER  
0  HIV pode ser transmitido de várias formas Marc...   USER  
1   "Eu não tenho nenhum tipo de contato com pess...   CHAT  


2